In [8]:
from PIL import Image
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision
from torchvision import transforms
from glob import glob
import re

In [4]:
pwd

'/mnt/nas100_vol2/LeeJungHoon/AOV_task(binary_clssification)/curation'

# external data에 대해서 curation 진행
- 파일 이름 정리
- cropping 진행
- 각각의 external과 internal data의 train/val/test datatset 합치기

In [5]:
external_abnormal_path = '/mnt/nas100_vol2/LeeJungHoon/bon_patients_image/external_abnormal_image/*.jpg'
exteranl_abnormal = glob(external_abnormal_path)
print(len(exteranl_abnormal))

189


In [6]:
exteranl_abnormal[0]

'/mnt/nas100_vol2/LeeJungHoon/bon_patients_image/external_abnormal_image/24523268 김민선 2018-06-28 (외부자료-저장) 내시경05d.jpg'

## external_abnormal(Positive) 이름 수정해서 파일 저장
- 파일 이름 : 환자번호_날짜_정보

In [ ]:
#external_abnormal(positive)
import shutil 

len_external_abnormal = len(exteranl_abnormal)



bin_external_abormal_matrix = []

for i in range(len_external_abnormal):
    
    c_abnormal = None
    d_abnormal = None
    e_abnormal = None
    f_abnormal = None
    
    file = exteranl_abnormal[i]
    
    #print(file)
    a_abnormal = file.split('/')[-1]
    b_abnormal = a_abnormal.split('(')[-1]

    c_abnormal = re.findall('05d+', b_abnormal)
    d_abnormal = re.findall('205d+', b_abnormal)
    e_abnormal = re.findall('305d+', b_abnormal)
    f_abnormal = re.findall('405d+', b_abnormal)
    g_abnormal = re.findall('505d+', b_abnormal)
    h_abnormal = re.findall('605d+', b_abnormal)
    j_abnormal = re.findall('705d+', b_abnormal)
    k_abnormal = re.findall('805d+', b_abnormal)
    l_abnormal = re.findall('905d+', b_abnormal)
    
#     print(c)
#     print(d)

    name_first = a_abnormal.split('(')[0]
#한글과 영어이름을 띄어쓰기로 변환하고, 띄어쓰기를 다시 _로 변환
    korean = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')
    name_first= re.sub(korean, '', name_first)
    name_first = re.sub('[a-zA-z]','',name_first).replace(' ', '_')
    name_first = re.sub(",", "", name_first)
    
# 마지막 글자가 다르거나 없는 경우가 존재하기 때문에 elif사용.    
    if d_abnormal:
        name_last=d_abnormal[0]
#         print(name_last)
        
    elif e_abnormal:
        name_last=e_abnormal[0]
        
    elif f_abnormal:
        name_last=f_abnormal[0]
        
    elif g_abnormal:
        name_last=g_abnormal[0]
    
    elif h_abnormal:
        name_last=h_abnormal[0]
    
    elif j_abnormal:
        name_last=j_abnormal[0]
    
    elif k_abnormal:
        name_last=k_abnormal[0]
        
    elif l_abnormal:
        name_last=l_abnormal[0]
        
    else:
        name_last=c_abnormal[0]
        

    external_abnormal_file_name = name_first + name_last+ '.jpg' 
    shutil.copy2(file,'/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/cropped_data/cropped_data_v4_ex_in/Negative_external_V4/%s' % external_abnormal_file_name)

    print(external_abnormal_file_name)
    bin_external_abormal_matrix.append(external_abnormal_file_name)
    



## external_abnormal(Positive) cropping 진행

In [10]:
# external_abnormal resize
from PIL import Image
import cv2

negative_path = '/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/cropped_data/cropped_data_v4_ex_in/Negative_external_V4/*.jpg' 
negative = glob(negative_path)
path_negative = '/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/cropped_data/cropped_data_v4_ex_in/Negative_external_crop_V4'


print(len(negative))


189


In [ ]:
err_list = []
for i in range(len(negative)):
    i = negative[i]
    image_name = i.split('/')[-1].split('.')[0]
    print(image_name)
    img = cv2.imread(i)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #들어오는 이미지의 컬러 형태가 BGR인지 RGB인지 모르기 때문에 변형



    image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_array = np.array(image_gray, dtype=np.uint8)
    edge = cv2.Canny(img_array,10,70)

    #vertical crop index
    len_width = len(edge[144,:])
    #vertical 방향으로 idx 확인
    img_index_left = []
    img_index_right = []

    for i in range(len_width):
        vertical_sum_left = np.sum(edge[:,i]/255)
        if vertical_sum_left > 140:
            img_index_left.append(i)

        vertical_sum_right = np.sum(edge[:,-(i+1)]/255)
        vertical_sum_end = np.sum(edge[:,-1]/255)
    
        if vertical_sum_end != 0:
            real_right_idx = 479
            img_index_right.append(real_right_idx)
        
        elif vertical_sum_right > 5:
            real_right_idx = len_width - (i+1) 
            img_index_right.append(real_right_idx)


    index_left = img_index_left[0]
    index_right = img_index_right[0]

    #horizontal crop index
    len_height = len(edge[:,144])
    #horizontal 방향으로 idx 확인
    img_index_up = []
    img_index_down = []
    for i in range(len_height):
        
        horizontal_sum_up = np.sum(edge[i,:]/255)
        horizontal_sum_start = np.sum(edge[0,:]/255)
        if horizontal_sum_start != 0:
            real_up_idx = 0
            img_index_up.append(i)
        elif horizontal_sum_up > 110:
            img_index_up.append(i)

        horizontal_sum_down = np.sum(edge[-(i+1),:]/255)
        horizontal_sum_end = np.sum(edge[-1,:]/255)
        

        if horizontal_sum_end != 0:
            real_down_idx = 639
            img_index_down.append(real_down_idx)
        elif horizontal_sum_down > 170:
            real_down_idx = len_height - (i+1) 
            img_index_down.append(real_down_idx)
       

    img_up = img_index_up[0]
    print(img_up)
    img_down = img_index_down[0]
    # print(img_index_down)
    print(img_down)

    img = img[img_up:img_down, index_left:index_right]

    print(img.shape)
    # print(np.max(img))

    # img = np.array(img, dtype = np.float32)
    # img = (img - np.min(img))/(np.max(img)-np.min(img))

    cv2.imwrite(path_negative+ '/'+image_name+'.jpg', img)
    # plt.imshow(img)
    # except:
    #     i = negative[i]
    #     image_name = i.split('/')[-1].split('.')[0]
    #     err_list.append(image_name)

In [ ]:
#try 사용
'''#normal resize
from PIL import Image
import cv2

negative_path = '/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/cropped_data/cropped_data_v4_ex_in/Negative_v4/*.jpg' 
negative = glob(negative_path)
path_negative = '/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/cropped_data/cropped_data_v4_ex_in/Negative_crop_v4'

err_list = []
for i in range(len(negative)):
    try:
        i = negative[i]
        image_name = i.split('/')[-1].split('.')[0]
        print(image_name)
        img = cv2.imread(i)
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #들어오는 이미지의 컬러 형태가 BGR인지 RGB인지 모르기 때문에 변형



        image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_array = np.array(image_gray, dtype=np.uint8)
        edge = cv2.Canny(img_array,50,200)

        #vertical crop index
        len_width = len(edge[144,:])
        #vertical 방향으로 idx 확인
        img_index_left = []
        img_index_right = []

        for i in range(len_width):
            vertical_sum_left = np.sum(edge[:,i]/255)
            if vertical_sum_left > 110:
                img_index_left.append(i)

            vertical_sum_right = np.sum(edge[:,-(i+1)]/255)
            if vertical_sum_right > 100:
                real_right_idx = len_width - (i+1) 
                img_index_right.append(real_right_idx) 

        index_left = img_index_left[0]
        index_right = img_index_right[0]

        #horizontal crop index
        len_height = len(edge[:,144])
        #horizontal 방향으로 idx 확인
        img_index_up = []
        img_index_down = []
        for i in range(len_height):
            horizontal_sum_up = np.sum(edge[i,:]/255)
            if horizontal_sum_up > 110:
                img_index_up.append(i)

            horizontal_sum_down = np.sum(edge[-(i+1),:]/255)
            horizontal_sum_end = np.sum(edge[-1,:]/255)

            if horizontal_sum_down > 100:
                if horizontal_sum_end != 0:
                    real_down_idx = 479
                    img_index_down.append(real_down_idx)    
                else:
                    real_down_idx = len_height - (i+1) 
                    img_index_down.append(real_down_idx)     

        img_up = img_index_up[0]
        print(img_up)
        img_down = img_index_down[0]
        print(img_index_down)
        print(img_down)

        img = img[img_up:img_down, index_left:index_right]

        print(img.shape)
        # print(np.max(img))

        # img = np.array(img, dtype = np.float32)
        # img = (img - np.min(img))/(np.max(img)-np.min(img))

        cv2.imwrite(path_negative+ '/'+image_name+'.jpg', img)
        # plt.imshow(img)
    except:
        i = negative[i]
        image_name = i.split('/')[-1].split('.')[0]
        err_list.append(image_name)
        '''

In [10]:
len(set(bin_matrix))

590

# internal data에 대해서 curation 진행
- 파일 이름 정리
- cropping 진행
- 각각의 external과 internal data의 train/val/test datatset 합치기

In [12]:
internal_abnormal_path = '/mnt/nas100_vol2/LeeJungHoon/bon_patients_image/Abnormal_image_v1/*.jpg'
interanl_abnormal = glob(internal_abnormal_path)
print(len(interanl_abnormal))

331


In [13]:
interanl_abnormal[0]

'/mnt/nas100_vol2/LeeJungHoon/bon_patients_image/Abnormal_image_v1/11182414 김동순 2019-01-17 식도위십이지장경검사 (비진정)05d.jpg'

## internal_abnormal(Positive) 이름 수정해서 파일 저장
- 파일 이름 : 환자번호_날짜_정보

In [ ]:
#external_abnormal(positive)
import shutil 

len_internal_abnormal = len(interanl_abnormal)



bin_internal_abormal_matrix = []

for i in range(len_internal_abnormal):
    
    c_abnormal = None
    d_abnormal = None
    e_abnormal = None
    f_abnormal = None
    
    file = interanl_abnormal[i]
    
    #print(file)
    a_abnormal = file.split('/')[-1]
    b_abnormal = a_abnormal.split('(')[-1]

    c_abnormal = re.findall('05d+', b_abnormal)
    d_abnormal = re.findall('205d+', b_abnormal)
    e_abnormal = re.findall('305d+', b_abnormal)
    f_abnormal = re.findall('405d+', b_abnormal)
    g_abnormal = re.findall('505d+', b_abnormal)
    h_abnormal = re.findall('605d+', b_abnormal)
    j_abnormal = re.findall('705d+', b_abnormal)
    k_abnormal = re.findall('805d+', b_abnormal)
    l_abnormal = re.findall('905d+', b_abnormal)
    
#     print(c)
#     print(d)

    name_first = a_abnormal.split('(')[0]
#한글과 영어이름을 띄어쓰기로 변환하고, 띄어쓰기를 다시 _로 변환
    korean = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')
    name_first= re.sub(korean, '', name_first)
    name_first = re.sub('[a-zA-z]','',name_first).replace(' ', '_')
    name_first = re.sub(",", "", name_first)
    
# 마지막 글자가 다르거나 없는 경우가 존재하기 때문에 elif사용.    
    if d_abnormal:
        name_last=d_abnormal[0]
#         print(name_last)
        
    elif e_abnormal:
        name_last=e_abnormal[0]
        
    elif f_abnormal:
        name_last=f_abnormal[0]
        
    elif g_abnormal:
        name_last=g_abnormal[0]
    
    elif h_abnormal:
        name_last=h_abnormal[0]
    
    elif j_abnormal:
        name_last=j_abnormal[0]
    
    elif k_abnormal:
        name_last=k_abnormal[0]
        
    elif l_abnormal:
        name_last=l_abnormal[0]
        
    else:
        name_last=c_abnormal[0]
        

    internal_abnormal_file_name = name_first + name_last+ '.jpg' 
    shutil.copy2(file,'/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/cropped_data/cropped_data_v4_ex_in/Negative_internal_V4/%s' % internal_abnormal_file_name)

    print(internal_abnormal_file_name)
    bin_external_abormal_matrix.append(internal_abnormal_file_name)
    



## internal_abnormal(Positive) cropping 진행

In [16]:
# external_abnormal resize
from PIL import Image
import cv2

negative_path = '/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/cropped_data/cropped_data_v4_ex_in/Negative_internal_V4/*.jpg' 
negative = glob(negative_path)
path_negative = '/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/cropped_data/cropped_data_v4_ex_in/Negative_internal_crop_V4'


print(len(negative))


331


In [ ]:
err_list = []
for i in range(len(negative)):
    i = negative[i]
    image_name = i.split('/')[-1].split('.')[0]
    print(image_name)
    img = cv2.imread(i)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #들어오는 이미지의 컬러 형태가 BGR인지 RGB인지 모르기 때문에 변형



    image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_array = np.array(image_gray, dtype=np.uint8)
    edge = cv2.Canny(img_array,10,70)

    #vertical crop index
    len_width = len(edge[144,:])
    #vertical 방향으로 idx 확인
    img_index_left = []
    img_index_right = []

    for i in range(len_width):
        vertical_sum_left = np.sum(edge[:,i]/255)
        if vertical_sum_left > 140:
            img_index_left.append(i)

        vertical_sum_right = np.sum(edge[:,-(i+1)]/255)
        vertical_sum_end = np.sum(edge[:,-1]/255)
    
        if vertical_sum_end != 0:
            real_right_idx = 479
            img_index_right.append(real_right_idx)
        
        elif vertical_sum_right > 5:
            real_right_idx = len_width - (i+1) 
            img_index_right.append(real_right_idx)


    index_left = img_index_left[0]
    index_right = img_index_right[0]

    #horizontal crop index
    len_height = len(edge[:,144])
    #horizontal 방향으로 idx 확인
    img_index_up = []
    img_index_down = []
    for i in range(len_height):
        
        horizontal_sum_up = np.sum(edge[i,:]/255)
        horizontal_sum_start = np.sum(edge[0,:]/255)
        if horizontal_sum_start != 0:
            real_up_idx = 0
            img_index_up.append(i)
        elif horizontal_sum_up > 110:
            img_index_up.append(i)

        horizontal_sum_down = np.sum(edge[-(i+1),:]/255)
        horizontal_sum_end = np.sum(edge[-1,:]/255)
        

        if horizontal_sum_end != 0:
            real_down_idx = 639
            img_index_down.append(real_down_idx)
        elif horizontal_sum_down > 170:
            real_down_idx = len_height - (i+1) 
            img_index_down.append(real_down_idx)
       

    img_up = img_index_up[0]
    print(img_up)
    img_down = img_index_down[0]
    # print(img_index_down)
    print(img_down)

    img = img[img_up:img_down, index_left:index_right]

    print(img.shape)
    # print(np.max(img))

    # img = np.array(img, dtype = np.float32)
    # img = (img - np.min(img))/(np.max(img)-np.min(img))

    cv2.imwrite(path_negative+ '/'+image_name+'.jpg', img)
    # plt.imshow(img)
    # except:
    #     i = negative[i]
    #     image_name = i.split('/')[-1].split('.')[0]
    #     err_list.append(image_name)

# data split 진행
- external / internal 따로 진행

In [21]:
ex_Abnormal_path = '/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/cropped_data/cropped_data_v4_ex_in/Negative_external_crop_V4/*.jpg'
ex_abnormal_glob = glob(ex_Abnormal_path)
abnormal_info =  pd.read_excel('/mnt/nas100_vol2/LeeJungHoon/AOV_task/patients_info_csv/220912_Patients_info.xlsx', sheet_name=2, engine='openpyxl')
df = abnormal_info['환자번호']
df_val = df.values
df_list = df_val.tolist()
df_str_list = list(map(str,df_list))
print(len(df_str_list))
print(len(set(df_str_list)))

69
65


In [24]:
ex_nor_img_list = []
for i in range(len(ex_abnormal_glob)):
    patient_num = ex_abnormal_glob[i].split('/')[-1].split('_')[0]
#     print(patient_num)
    ex_nor_img_list.append(patient_num)
ex_one_nor_img_list = list(set(ex_nor_img_list))
# ex_one_nor_img_list
len(ex_one_nor_img_list)

67

In [27]:
# positive(abnormal)
ex_img_no_csv_list = []
ex_img_csv_list = []
for i in ex_one_nor_img_list:
#     print(i)
#     patient_num_img = abnormal_glob[i].split('/')[-1].split('_')[0]

    if i not in df_str_list:
        ex_img_no_csv_list.append(i)
#         print('파일이 존재하지 않습니다:',i)
    else:
        ex_img_csv_list.append(i)
#         print('파일이 존재:',i)

#     else:
#         print('있습니다!!')
        
print(len(ex_img_no_csv_list))
print(len(ex_img_csv_list))
ex_img_no_csv_list
# print(len(img_csv_list))
# print(len(no_list))

4
63


['50462603', '56998629', '57443920', '56141548']

In [26]:
# positive(abnormal)
ex_csv_no_img_list = []
ex_img_csv_list = []
for i in df_str_list:
#     print(i)
#     patient_num_img = abnormal_glob[i].split('/')[-1].split('_')[0]

    if i not in ex_one_nor_img_list:
        ex_csv_no_img_list.append(i)
#         print('파일이 존재하지 않습니다:',i)
    else:
        ex_img_csv_list.append(i)
#         print('파일이 존재:',i)

#     else:
#         print('있습니다!!')
        
print(len(ex_csv_no_img_list))
print(len(ex_img_csv_list))
ex_csv_no_img_list
# print(len(img_csv_list))
# print(len(no_list))

2
67


['56426629', '61437986']

In [28]:
ex_ab_no_file = ex_img_no_csv_list + ex_csv_no_img_list
print(len(ex_ab_no_file))

6


In [30]:
# normal 파일에서 누락된 이미지 제거 후 새로운 파일에 저장
b = []
c = []
for i in range(len(ex_abnormal_glob)):
    patient_num = ex_abnormal_glob[i].split('/')[-1].split('_')[0]
    name = ex_abnormal_glob[i].split('/')[-1]

    a = ex_abnormal_glob[i]
    
#     print(a)
    if patient_num not in ex_ab_no_file:
            b.append(patient_num)
            shutil.copy2(a,'/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v4/external_abn_data_v4/%s' %name)
#     else:
#         c.append(patient_num)
        


In [49]:
ex_abn_path = '/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v4/external_abn_data_v4/*.jpg'
ex_abn_data = glob(ex_abn_path)
print(len(ex_abn_data))
"""
현재 엑셀 파일에는 존재하지만 이미지가 존재하지 않는 환자의 수는 4명이지만 
4명의 총 이미지 파일 수는 13장이기 때문에 총 189장 중 176장만 존재
176
"""
in_abn_path = '/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v4/internal_abn_data_v4/*.jpg'
in_abn_data = glob(in_abn_path)
print(len(in_abn_data))

176
331


## ex_abnormal split

In [50]:
ex_abn_data_id = []
in_abn_data_id = []

for i in range(len(ex_abn_data)):
    ex_abn_data_id.append(ex_abn_data[i].split('/')[-1].split('_')[0])
    

for j in range(len(in_abn_data)):
    in_abn_data_id.append(in_abn_data[j].split('/')[-1].split('_')[0])
    
#이미지수 확인    
print(len(ex_abn_data_id),len(in_abn_data_id)) 

176 331


In [51]:
#patients 수 확인
ex_abn_one_id = list(set(ex_abn_data_id))
in_abn_one_id = list(set(in_abn_data_id))

print(len(ex_abn_one_id),len(in_abn_one_id))

63 123


In [ ]:
ex_abn_one_id

In [53]:
#external
from sklearn.model_selection import train_test_split

ex_x_train, ex_x_test = train_test_split(ex_abn_one_id,test_size=0.3,random_state=42)
ex_x_test, ex_x_valid = train_test_split(ex_x_test,test_size=0.5,random_state=42)

In [54]:
print(len(ex_x_train))
print(len(ex_x_valid))
print(len(ex_x_test))

44
10
9


In [55]:
train = []
val =[]
test= []

for i in range(len(ex_abn_data)):
    
    patient_num = ex_abn_data[i].split('/')[-1].split('_')[0]
    name = ex_abn_data[i].split('/')[-1]

    a = ex_abn_data[i]
    # /mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v4/exist_abn_data_v4/*.jpg'
#     print(a)
    if patient_num in ex_x_train:
            train.append(patient_num)
            shutil.copy2(a,'/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v4/data_v4_7_1/abnormal_V4/ex_abnormal_train_v4/%s' %name)
            
    elif patient_num in ex_x_valid:
            val.append(patient_num)
            shutil.copy2(a,'/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v4/data_v4_7_1/abnormal_V4/ex_abnormal_valid_v4/%s' %name) 
            
    elif patient_num in ex_x_test:
            test.append(patient_num)
            shutil.copy2(a,'/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v4/data_v4_7_1/abnormal_V4/ex_abnormal_test_v4/%s' %name)
print(len(train),len(val), len(test))

118 27 31


## in_abnormal split

In [57]:
#internal
from sklearn.model_selection import train_test_split
in_x_train, in_x_test = train_test_split(in_abn_one_id,test_size=0.3,random_state=42)
in_x_test, in_x_valid = train_test_split(in_x_test,test_size=0.5,random_state=42)

In [58]:
train = []
val =[]
test= []

for i in range(len(in_abn_data)):
    patient_num = in_abn_data[i].split('/')[-1].split('_')[0]
    name = in_abn_data[i].split('/')[-1]

    a = in_abn_data[i]
    # /mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v4/exist_abn_data_v4/*.jpg'
#     print(a)
    if patient_num in in_x_train:
            train.append(patient_num)
            shutil.copy2(a,'/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v4/data_v4_7_1/abnormal_V4/in_abnormal_train_v4/%s' %name)
            
    elif patient_num in in_x_valid:
            val.append(patient_num)
            shutil.copy2(a,'/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v4/data_v4_7_1/abnormal_V4/in_abnormal_valid_v4/%s' %name) 
            
    elif patient_num in in_x_test:
            test.append(patient_num)
            shutil.copy2(a,'/mnt/nas100_vol2/LeeJungHoon/Aov_task_curation_data/curation_data_v4/data_v4_7_1/abnormal_V4/in_abnormal_test_v4/%s' %name)
print(len(train),len(val), len(test))

236 48 47


In [59]:
print(len(ex_x_train),len(ex_x_valid),len(ex_x_test))
print(len(in_x_train),len(in_x_valid),len(in_x_test))

44 10 9
86 19 18
